In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [2]:
def pepper_salt_noise(p,image):
    m, n = image.shape
    frac = 1- p
    sample = np.random.binomial(1, frac, size=m*n)
    mask = np.reshape(sample, (m, n))
    image_noise = image * mask
    return mask, image_noise

In [3]:
def rgb2gray(image):
    r, g, b = image[:,:,0], image[:,:,1], image[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [4]:
# example
image = cv2.imread('persian_girl.jpg')
image_gs =  rgb2gray(image)
cv2.imwrite("gs.jpg", image_gs)

mask10, image_noise10 =  pepper_salt_noise(0.1,image_gs)
cv2.imwrite("gs_noisy_10.jpg", image_noise10 )

mask20, image_noise20 =  pepper_salt_noise(0.2,image_gs)
cv2.imwrite("gs_noisy_20.jpg", image_noise20 )

mask30, image_noise30 =  pepper_salt_noise(0.3,image_gs)
cv2.imwrite("gs_noisy_30.jpg", image_noise30)

mask40, image_noise40 =  pepper_salt_noise(0.4,image_gs)
cv2.imwrite("gs_noisy_40.jpg", image_noise40)

mask50, image_noise50 =  pepper_salt_noise(0.5,image_gs)
cv2.imwrite("gs_noisy_50.jpg", image_noise50)

mask60, image_noise60 =  pepper_salt_noise(0.6,image_gs)
cv2.imwrite("gs_noisy_50.jpg", image_noise60)

True

In [5]:
# https://github.com/vincenzobaz/RobustPCA/blob/master/code/robust_pca.py
    
def l1_norm(mat):
    return np.sum(np.abs(mat))


def frob_norm(mat):
    return np.sqrt(np.sum(np.power(mat, 2)))


def shrinkage(tau, X):
    return np.sign(X) * np.maximum(np.abs(X) - tau, 0)


def thresholding(tau, X):
    u, s, vh = np.linalg.svd(X, full_matrices=False, compute_uv=True)
    s = shrinkage(tau, s)
    return np.multiply(u, s) @ vh


def robust_pca(M, max_iters, delta=10**(-7)):
    h, w = M.shape
    S = np.zeros(M.shape)
    Y = np.zeros(M.shape)
    L = None
    m_norm = frob_norm(M)
    mu = h * w / (4 * l1_norm(M))
    #mu = 0.001
    muinv = 1 / mu
    lambda_ = 1 / np.sqrt(max(h, w))

    converged = False
    n_iter = 0
    while n_iter < max_iters and not converged:
        muinvY = muinv * Y
        L = thresholding(muinv, M - S + muinvY)
        S = shrinkage(lambda_ * muinv, M - L + muinvY)
        Y = Y + mu * (M - L - S)

        converged = frob_norm(M - L - S) < delta * m_norm
        n_iter += 1

    print(f'Completed with {n_iter} SVDs')
    return L, S, n_iter



In [6]:
L10, S10, n_iter10 = robust_pca(image_noise10, 1000, delta=10**(-7))
L20, S20, n_iter20 = robust_pca(image_noise20, 1000, delta=10**(-7))
L30, S30, n_iter30 = robust_pca(image_noise30, 1000, delta=10**(-7))
L40, S40, n_iter40 = robust_pca(image_noise40, 1000, delta=10**(-7))
L50, S50, n_iter50 = robust_pca(image_noise50, 1000, delta=10**(-7))
L60, S60, n_iter60 = robust_pca(image_noise60, 1000, delta=10**(-7))



Completed with 1000 SVDs
Completed with 586 SVDs
Completed with 292 SVDs
Completed with 702 SVDs
Completed with 813 SVDs
Completed with 292 SVDs


In [7]:
cv2.imwrite("recovery10.jpg", L10 )
cv2.imwrite("recovery20.jpg", L20 )
cv2.imwrite("recovery30.jpg", L30 )
cv2.imwrite("recovery40.jpg", L40 )
cv2.imwrite("recovery50.jpg", L50 )
cv2.imwrite("recovery60.jpg", L60 )


True

In [18]:
from skimage.measure import compare_ssim as ssim
ssim(image_gs,L60)

C:\Users\hamid\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  


8.07728344632673e-06